In [ ]:
import os
import zipfile
import shutil
import pandas as pd
import re

# 경로 설정
input_dir = r"D:\Landslide\data\raw\sentinel-2\gyeongnam\wildfire_pre_match_2016_2020_buf15_cloud40"
output_dir = r"D:\Landslide\data\raw\sentinel-2\gyeongnam\pre_unzip"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def extract_short_name(filename):
    """Sentinel-2 파일명에서 고유 식별자 추출"""
    # S2A_MSIL2A_20151028T020802_N0500_R103_T52SDE_20231009T132010.SAFE.zip
    # -> 20151028_R103_T52SDE (날짜_궤도_타일)
    
    # 전체 패턴 매칭
    match = re.search(r'(\d{8})T\d+_N\d+_(R\d+)_(T\w+)_', filename)
    if match:
        date = match.group(1)
        orbit = match.group(2)
        tile = match.group(3)
        return f"{date}_{orbit}_{tile}"
    
    # 실패시 원본 파일명 사용 (확장자만 제거)
    return filename.replace('.zip', '').replace('.SAFE', '')

print(f"=== Sentinel-2 압축 파일 해제 시작 ===")
print(f"입력 경로: {input_dir}")
print(f"출력 경로: {output_dir}\n")

success_count = 0
skip_count = 0
fail_count = 0
failed_files = []

# ZIP 파일 목록 가져오기
all_zip_files = []
for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.endswith('.zip'):
            all_zip_files.append((root, file))

print(f"발견된 ZIP 파일: {len(all_zip_files)}개\n")

for idx, (root, file) in enumerate(all_zip_files, 1):
    zip_path = os.path.join(root, file)
    
    # 짧은 이름 생성
    short_name = extract_short_name(file)
    output_path = os.path.join(output_dir, short_name)
    
    print(f"[{idx}/{len(all_zip_files)}] {short_name}")
    
    # GRANULE 폴더 확인
    granule_check = os.path.join(output_path, "GRANULE")
    
    if os.path.exists(output_path):
        if not os.path.exists(granule_check):
            print(f"  ⚠️  불완전한 압축 해제, 삭제 후 재시도")
            try:
                shutil.rmtree(output_path)
            except Exception as e:
                print(f"  ✗ 폴더 삭제 실패: {e}")
                fail_count += 1
                failed_files.append((file, f"폴더 삭제 실패: {e}"))
                continue
        else:
            print(f"  - 이미 압축 해제됨")
            skip_count += 1
            continue
    
    # 압축 해제
    try:
        print(f"  ▶ 압축 해제 중...")
        
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_contents = zip_ref.namelist()
            common_prefix = os.path.commonprefix(zip_contents)
            if common_prefix and '/' in common_prefix:
                common_prefix = common_prefix.split('/')[0]
            
            temp_dir = output_path + "_temp"
            if os.path.exists(temp_dir):
                shutil.rmtree(temp_dir)
            
            zip_ref.extractall(temp_dir)
            
            # 중복 폴더 구조 평탄화
            if common_prefix and os.path.isdir(os.path.join(temp_dir, common_prefix)):
                nested_path = os.path.join(temp_dir, common_prefix)
                shutil.move(nested_path, output_path)
                shutil.rmtree(temp_dir)
            else:
                shutil.move(temp_dir, output_path)
        
        # GRANULE 폴더 검증
        if os.path.exists(granule_check):
            print(f"  ✓ 완료")
            success_count += 1
        else:
            print(f"  ✗ GRANULE 폴더 없음")
            fail_count += 1
            failed_files.append((file, "GRANULE 폴더 없음"))
            if os.path.exists(output_path):
                shutil.rmtree(output_path)
                
    except Exception as e:
        error_msg = str(e)[:150]
        print(f"  ✗ 오류: {error_msg}")
        fail_count += 1
        failed_files.append((file, error_msg))
        for cleanup_path in [output_path, output_path + "_temp"]:
            if os.path.exists(cleanup_path):
                try:
                    shutil.rmtree(cleanup_path)
                except:
                    pass

print(f"\n{'='*60}")
print(f"=== 압축 해제 완료 ===")
print(f"총 파일: {len(all_zip_files)}개")
print(f"성공: {success_count}개")
print(f"건너뜀: {skip_count}개")
print(f"실패: {fail_count}개")
print(f"출력 경로: {output_dir}")

if failed_files:
    print(f"\n{'='*60}")
    print(f"=== 실패한 파일 ({len(failed_files)}개) ===")
    for i, (filename, error) in enumerate(failed_files[:10], 1):
        print(f"{i}. {filename[:60]}...")
        print(f"   오류: {error}")
    
    if len(failed_files) > 10:
        print(f"\n... 외 {len(failed_files) - 10}개 더")
    
    # 실패 목록 CSV 저장
    fail_csv = os.path.join(output_dir, "failed_files.csv")
    df_fail = pd.DataFrame(failed_files, columns=['filename', 'error'])
    df_fail.to_csv(fail_csv, index=False, encoding='utf-8-sig')
    print(f"\n실패 목록 저장: {fail_csv}")

=== Sentinel-2 압축 파일 해제 시작 ===
입력 경로: D:\Landslide\data\sentinel-2\wildfire_pre_match_2016_2020_buf15_cloud40
출력 경로: D:\Landslide\data\sentinel-2\pre_unzip

발견된 ZIP 파일: 128개

[1/128] 20151028_R103_T52SDE
  - 이미 압축 해제됨
[2/128] 20151028_R103_T52SEE
  - 이미 압축 해제됨
[3/128] 20151217_R103_T52SDD
  - 이미 압축 해제됨
[4/128] 20170130_R103_T52SDE
  - 이미 압축 해제됨
[5/128] 20170130_R103_T52SEE
  - 이미 압축 해제됨
[6/128] 20170202_R003_T52SCE
  - 이미 압축 해제됨
[7/128] 20170209_R103_T52SDD
  - 이미 압축 해제됨
[8/128] 20170212_R003_T52SCE
  - 이미 압축 해제됨
[9/128] 20170304_R003_T52SCD
  - 이미 압축 해제됨
[10/128] 20170304_R003_T52SCE
  - 이미 압축 해제됨
[11/128] 20170304_R003_T52SDD
  - 이미 압축 해제됨
[12/128] 20170304_R003_T52SDE
  - 이미 압축 해제됨
[13/128] 20170321_R103_T52SDE
  - 이미 압축 해제됨
[14/128] 20170410_R103_T52SDE
  - 이미 압축 해제됨
[15/128] 20170413_R003_T52SDE
  - 이미 압축 해제됨
[16/128] 20170430_R103_T52SCE
  - 이미 압축 해제됨
[17/128] 20170503_R003_T52SDE
  - 이미 압축 해제됨
[18/128] 20170609_R103_T52SDE
  - 이미 압축 해제됨
[19/128] 20170719_R103_T52SEE
  - 이미 압축 해제

In [ ]:
# BAIS2 계산 셀
import rasterio
import numpy as np
import glob
import os
import xml.etree.ElementTree as ET
from rasterio.warp import reproject, Resampling

# 첫 번째 셀의 output_dir와 일치시킴
input_dir = r"D:\Landslide\data\raw\sentinel-2\gyeongnam\pre_unzip"
output_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\pre\BAIS2"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 마스킹할 클래스들 (구름, 해역, 그림자 등)
invalid_classes = [0, 1, 3, 6, 8, 9, 10]

def extract_boa_offset(safe_folder):
    """MTD_MSIL2A.xml에서 BOA_ADD_OFFSET 추출"""
    metadata_file = os.path.join(safe_folder, "MTD_MSIL2A.xml")
    
    if not os.path.exists(metadata_file):
        return 0
    
    try:
        tree = ET.parse(metadata_file)
        root = tree.getroot()
        
        # BOA_ADD_OFFSET 찾기
        for elem in root.iter():
            if 'BOA_ADD_OFFSET' in elem.tag:
                offset_text = elem.text.strip()
                if offset_text:
                    return int(offset_text)
        
        return 0
        
    except Exception as e:
        return 0

def load_band(granule_path, band_name, resolution='20m'):
    """특정 밴드 로드 (R20m 폴더 우선)"""
    img_data_path = os.path.join(granule_path, "IMG_DATA")
    
    # R20m 폴더에서 찾기
    res_folder = os.path.join(img_data_path, f"R{resolution}")
    
    if os.path.exists(res_folder):
        pattern = os.path.join(res_folder, f"*_{band_name}_{resolution}.jp2")
        files = glob.glob(pattern)
        if files:
            return files[0]
    
    # 해상도 표시 없는 파일명 시도
    pattern = os.path.join(img_data_path, "**", f"*_{band_name}.jp2")
    files = glob.glob(pattern, recursive=True)
    if files:
        return files[0]
    
    return None

def calculate_bais2(safe_folder):
    """단일 SAFE 폴더에 대해 BAIS2 계산"""
    folder_name = os.path.basename(safe_folder)
    print(f"\n처리 중: {folder_name}")
    
    # GRANULE 폴더 찾기
    granule_pattern = os.path.join(safe_folder, "GRANULE", "*")
    granule_dirs = glob.glob(granule_pattern)
    
    if not granule_dirs:
        print(f"  ✗ GRANULE 폴더 없음")
        return False
    
    granule_path = granule_dirs[0]
    
    # BOA_ADD_OFFSET 추출
    boa_offset = extract_boa_offset(safe_folder)
    if boa_offset != 0:
        print(f"  📋 BOA_ADD_OFFSET: {boa_offset}")
    
    # 필요한 밴드 파일 경로 찾기
    bands_needed = {
        'B04': 'B04',
        'B06': 'B06', 
        'B07': 'B07',
        'B8A': 'B8A',
        'B12': 'B12',
        'SCL': 'SCL'
    }
    
    band_files = {}
    for key, band_name in bands_needed.items():
        file_path = load_band(granule_path, band_name, '20m')
        if file_path:
            band_files[key] = file_path
        else:
            print(f"  ✗ {band_name}: 파일 없음")
            return False
    
    if len(band_files) != len(bands_needed):
        print(f"  ✗ 필요한 밴드가 모두 없음")
        return False
    
    # 밴드 데이터 읽기
    with rasterio.open(band_files['B04']) as src:
        b4 = src.read(1).astype(np.float32)
        crs = src.crs
        transform = src.transform
        width = src.width
        height = src.height
    
    with rasterio.open(band_files['B06']) as src:
        b6 = src.read(1).astype(np.float32)
    
    with rasterio.open(band_files['B07']) as src:
        b7 = src.read(1).astype(np.float32)
    
    with rasterio.open(band_files['B8A']) as src:
        b8a = src.read(1).astype(np.float32)
    
    with rasterio.open(band_files['B12']) as src:
        b12 = src.read(1).astype(np.float32)
    
    with rasterio.open(band_files['SCL']) as src:
        scl = src.read(1)
    
    # BOA offset 적용 및 정규화 (DN -> 반사도)
    quantification_value = 10000
    b4 = (b4 + boa_offset) / quantification_value
    b6 = (b6 + boa_offset) / quantification_value
    b7 = (b7 + boa_offset) / quantification_value
    b8a = (b8a + boa_offset) / quantification_value
    b12 = (b12 + boa_offset) / quantification_value
    
    # 음수 값을 0으로 클리핑 (물리적으로 불가능한 반사도 제거)
    b4 = np.clip(b4, 0, None)
    b6 = np.clip(b6, 0, None)
    b7 = np.clip(b7, 0, None)
    b8a = np.clip(b8a, 0, None)
    b12 = np.clip(b12, 0, None)
    
    # 0으로 나누기 방지를 위한 epsilon
    epsilon = 1e-6
    
    # === 개선된 마스킹 로직 ===
    # 1. SCL 기반 마스크 (구름, 그림자, 물 등)
    scl_mask = np.isin(scl, invalid_classes)
    
    # 2. Red 밴드(b4)가 매우 작은 픽셀 마스크 (division by near-zero 방지)
    b4_invalid_mask = b4 < epsilon
    
    # 3. 분모가 0에 가까운 경우 마스크
    b12_b8a_sum = b12 + b8a
    b12_b8a_invalid_mask = b12_b8a_sum < epsilon
    
    # BAIS2 계산 (마스킹 전)
    # 분모를 epsilon으로 대체하여 계산
    b4_safe = np.where(b4 < epsilon, epsilon, b4)
    b12_b8a_sum_safe = np.where(b12_b8a_sum < epsilon, epsilon, b12_b8a_sum)
    
    term1 = 1 - np.sqrt((b6 * b7 * b8a) / b4_safe)
    term2 = (b12 - b8a) / np.sqrt(b12_b8a_sum_safe) + 1
    bais2 = term1 * term2
    
    # 4. inf 또는 nan 값 마스크
    invalid_values_mask = ~np.isfinite(bais2)
    
    # 5. 최종 마스크 통합
    final_mask = scl_mask | b4_invalid_mask | b12_b8a_invalid_mask | invalid_values_mask
    
    # 최종 마스킹 적용
    bais2[final_mask] = np.nan
    
    # 마스킹 통계
    total_pixels = final_mask.size
    scl_masked = np.sum(scl_mask)
    b4_masked = np.sum(b4_invalid_mask)
    b12_b8a_masked = np.sum(b12_b8a_invalid_mask)
    invalid_masked = np.sum(invalid_values_mask)
    total_masked = np.sum(final_mask)
    masked_ratio = (total_masked / total_pixels) * 100
    
    # GeoTIFF로 저장 (GTiff 드라이버 명시)
    output_filename = f"{folder_name}_BAIS2.tif"
    output_path = os.path.join(output_dir, output_filename)
    
    # GeoTIFF 프로파일 설정
    profile = {
        'driver': 'GTiff',
        'dtype': rasterio.float32,
        'count': 1,
        'width': width,
        'height': height,
        'crs': crs,
        'transform': transform,
        'compress': 'lzw',
        'nodata': np.nan
    }
    
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(bais2, 1)
        dst.set_band_description(1, 'BAIS2')
        dst.update_tags(1, 
                       BAND_NAME='BAIS2',
                       DESCRIPTION='Burned Area Index for Sentinel-2',
                       FORMULA='(1 - sqrt((B6*B7*B8A)/B4)) * ((B12-B8A)/sqrt(B12+B8A) + 1)',
                       BOA_OFFSET=str(boa_offset),
                       TOTAL_MASKED_RATIO=f'{masked_ratio:.1f}%',
                       SCL_MASKED_PIXELS=str(scl_masked),
                       B4_INVALID_PIXELS=str(b4_masked),
                       B12_B8A_INVALID_PIXELS=str(b12_b8a_masked),
                       INVALID_VALUES_PIXELS=str(invalid_masked))
    
    print(f"  ✅ {output_filename}")
    print(f"     BAIS2: {np.nanmin(bais2):.3f} ~ {np.nanmax(bais2):.3f}")
    print(f"     마스킹: {masked_ratio:.1f}% (SCL:{scl_masked}, B4:{b4_masked}, B12+B8A:{b12_b8a_masked}, Invalid:{invalid_masked})")
    
    return True

# 메인 실행
print(f"=== BAIS2 계산 시작 ===")
print(f"입력 경로: {input_dir}")
print(f"출력 경로: {output_dir}\n")

# 모든 SAFE 폴더 찾기
safe_folders = [d for d in glob.glob(os.path.join(input_dir, "*")) if os.path.isdir(d)]
print(f"발견된 SAFE 폴더: {len(safe_folders)}개\n")

success_count = 0
fail_count = 0
failed_list = []

for idx, safe_folder in enumerate(safe_folders, 1):
    try:
        print(f"[{idx}/{len(safe_folders)}]", end=" ")
        if calculate_bais2(safe_folder):
            success_count += 1
        else:
            fail_count += 1
            failed_list.append(os.path.basename(safe_folder))
    except Exception as e:
        print(f"  ✗ 오류: {str(e)[:100]}")
        fail_count += 1
        failed_list.append(os.path.basename(safe_folder))

print(f"\n{'='*60}")
print(f"=== BAIS2 계산 완료 ===")
print(f"성공: {success_count}개")
print(f"실패: {fail_count}개")
print(f"출력 경로: {output_dir}")

if failed_list:
    print(f"\n실패 목록 ({len(failed_list)}개):")
    for i, name in enumerate(failed_list[:10], 1):
        print(f"  {i}. {name}")
    if len(failed_list) > 10:
        print(f"  ... 외 {len(failed_list) - 10}개")

=== BAIS2 계산 시작 ===
입력 경로: D:\Landslide\data\sentinel-2\pre_unzip
출력 경로: D:\Landslide\data\sentinel-2\outputs\pre\BAIS2

발견된 SAFE 폴더: 128개

[1/128] 
처리 중: 20151028_R103_T52SDE
  📋 BOA_ADD_OFFSET: -1000
  ✅ 20151028_R103_T52SDE_BAIS2.tif
     BAIS2: -7.326 ~ 2.106
     마스킹: 3.4% (SCL:1026123, B4:2885, B12+B8A:4, Invalid:0)
[2/128] 
처리 중: 20151028_R103_T52SEE
  📋 BOA_ADD_OFFSET: -1000
  ✅ 20151028_R103_T52SEE_BAIS2.tif
     BAIS2: -1.690 ~ 2.115
     마스킹: 66.7% (SCL:20099584, B4:16895, B12+B8A:330, Invalid:0)
[3/128] 
처리 중: 20151217_R103_T52SDD
  📋 BOA_ADD_OFFSET: -1000
  ✅ 20151217_R103_T52SDD_BAIS2.tif
     BAIS2: -3.118 ~ 1.985
     마스킹: 71.9% (SCL:21671251, B4:49653, B12+B8A:89531, Invalid:0)
[4/128] 
처리 중: 20170130_R103_T52SDE
  📋 BOA_ADD_OFFSET: -1000
  ✅ 20170130_R103_T52SDE_BAIS2.tif
     BAIS2: -7.511 ~ 1.779
     마스킹: 16.2% (SCL:4856362, B4:19844, B12+B8A:3, Invalid:0)
[5/128] 
처리 중: 20170130_R103_T52SEE
  📋 BOA_ADD_OFFSET: -1000
  ✅ 20170130_R103_T52SEE_BAIS2.tif
     BAIS2: -

In [ ]:
# 위의 셀에서 생성된 BAIS2의 tif파일들의 영상 각각에 대해 히스토그램(빈도)를 계산하고, 150개 히스토그램을 종합하여, 각 영상의 히스토그램이 서로 얼마나 다른 분포를 가지는지, 분석하는 셀
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
from scipy.spatial.distance import jensenshannon
from scipy.stats import wasserstein_distance
from tqdm import tqdm

# 한글 폰트 설정 (Windows 환경)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

input_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\pre\BAIS2"
analysis_output_dir = os.path.join(os.path.dirname(input_dir), "BAIS2_analysis")

if not os.path.exists(analysis_output_dir):
    os.makedirs(analysis_output_dir)

print("=" * 70)
print("=== BAIS2 히스토그램 분포 분석 시작 ===")
print("=" * 70)
print(f"입력 경로: {input_dir}")
print(f"분석 결과 저장: {analysis_output_dir}\n")

# ============================================================
# 1단계: 데이터 수집 및 전처리 (메모리 효율적)
# ============================================================
print("[1/5] 데이터 수집 및 기본 통계 계산 중...")

tif_files = sorted(glob.glob(os.path.join(input_dir, "*.tif")))
print(f"      발견된 BAIS2 파일: {len(tif_files)}개\n")

if len(tif_files) == 0:
    print("❌ 분석할 TIF 파일이 없습니다.")
    raise FileNotFoundError("No TIF files found")

# 기본 통계 수집 (메모리 효율적)
statistics_list = []
sample_per_image = 2000  # 각 영상당 샘플링할 픽셀 수 (더 작게)

print(f"      메모리 효율을 위해 각 영상당 {sample_per_image:,}개 픽셀만 샘플링합니다.\n")

for tif_path in tqdm(tif_files, desc="      통계 수집"):
    filename = os.path.basename(tif_path)
    
    with rasterio.open(tif_path) as src:
        data = src.read(1)
        
        # 유효한 픽셀만 추출 (NaN 제외)
        valid_data = data[~np.isnan(data)]
        
        if len(valid_data) > 0:
            stats = {
                'filename': filename,
                'min': float(np.min(valid_data)),
                'max': float(np.max(valid_data)),
                'mean': float(np.mean(valid_data)),
                'median': float(np.median(valid_data)),
                'std': float(np.std(valid_data)),
                'valid_pixels': len(valid_data),
                'total_pixels': data.size,
                'valid_ratio': len(valid_data) / data.size
            }
            statistics_list.append(stats)

df_stats = pd.DataFrame(statistics_list)
total_valid_pixels = df_stats['valid_pixels'].sum()

print(f"\n✅ 통계 수집 완료: {len(df_stats)}개 영상")
print(f"   전체 유효 픽셀 수: {total_valid_pixels:,}개")
print(f"   BAIS2 전체 범위: [{df_stats['min'].min():.3f}, {df_stats['max'].max():.3f}]")
print(f"   평균의 평균: {df_stats['mean'].mean():.3f} ± {df_stats['mean'].std():.3f}")
print(f"   표준편차의 평균: {df_stats['std'].mean():.3f}\n")

# ============================================================
# 2단계: 히스토그램 계산
# ============================================================
print("[2/5] 히스토그램 계산 중...")

# 공통 bin 범위 설정 (전체 데이터 기준)
global_min = df_stats['min'].min()
global_max = df_stats['max'].max()
n_bins = 50  # bins 수를 줄여서 메모리 절약
bins = np.linspace(global_min, global_max, n_bins + 1)
bin_centers = (bins[:-1] + bins[1:]) / 2

print(f"      Bins: {n_bins}개 (범위: [{global_min:.3f}, {global_max:.3f}])")

# 각 영상별 히스토그램 계산
histograms = []  # 정규화된 히스토그램 저장
histogram_counts = []  # 원본 카운트 저장

for tif_path in tqdm(tif_files, desc="      히스토그램 계산"):
    with rasterio.open(tif_path) as src:
        data = src.read(1)
        valid_data = data[~np.isnan(data)]
        
        if len(valid_data) > 0:
            # 히스토그램 계산
            counts, _ = np.histogram(valid_data, bins=bins)
            histogram_counts.append(counts)
            
            # 정규화 (확률 분포로 변환)
            normalized = counts / counts.sum()
            histograms.append(normalized)

histograms = np.array(histograms)
histogram_counts = np.array(histogram_counts)

print(f"\n✅ 히스토그램 계산 완료: {histograms.shape[0]}개 영상\n")

# ============================================================
# 3단계: 분포 차이 분석
# ============================================================
print("[3/5] 분포 차이 분석 중...")

# 3-1. 통계적 변동성
mean_cv = df_stats['mean'].std() / df_stats['mean'].mean()  # 변동 계수
std_cv = df_stats['std'].std() / df_stats['std'].mean()

print(f"      평균의 변동 계수 (CV): {mean_cv:.4f}")
print(f"      표준편차의 변동 계수 (CV): {std_cv:.4f}")

# 3-2. 평균 히스토그램 계산
mean_histogram = histograms.mean(axis=0)
std_histogram = histograms.std(axis=0)

# 3-3. Jensen-Shannon Divergence 계산 (샘플링: 계산량 감소)
print(f"\n      분포 거리 계산 중 (샘플링)...")
n_samples = min(len(histograms), 30)  # 샘플 수를 더 줄임
sample_indices = np.random.choice(len(histograms), n_samples, replace=False)

js_divergences = []

for i in tqdm(range(n_samples), desc="      JS Divergence"):
    idx = sample_indices[i]
    # 평균 분포와의 거리
    js_dist = jensenshannon(histograms[idx], mean_histogram)
    js_divergences.append(js_dist)

print(f"\n✅ 분포 차이 분석 완료")
print(f"   평균 JS Divergence: {np.mean(js_divergences):.4f} ± {np.std(js_divergences):.4f}\n")

# ============================================================
# 4단계: 시각화
# ============================================================
print("[4/5] 시각화 생성 중...")

fig = plt.figure(figsize=(16, 10))  # 크기를 줄임
gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)

# 4-1. 전체 통합 히스토그램 (개별 히스토그램 합산)
ax1 = fig.add_subplot(gs[0, :])
# 모든 영상의 히스토그램을 합산하여 전체 분포 표현
total_histogram = histogram_counts.sum(axis=0)
ax1.bar(bin_centers, total_histogram, width=(bins[1]-bins[0])*0.9, color='steelblue', alpha=0.7, edgecolor='black')
ax1.set_xlabel('BAIS2 값', fontsize=12)
ax1.set_ylabel('빈도 (픽셀 수)', fontsize=12)
ax1.set_title(f'전체 BAIS2 분포 통합 히스토그램 ({len(tif_files)}개 영상, {total_valid_pixels:,}개 픽셀)', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)
ax1.axvline(df_stats['mean'].mean(), color='red', linestyle='--', linewidth=2, label=f'전체 평균: {df_stats["mean"].mean():.3f}')
ax1.legend(fontsize=11)

# 4-2. 개별 히스토그램 오버레이 (샘플)
ax2 = fig.add_subplot(gs[1, 0])
n_overlay = min(20, len(histograms))  # 샘플 수 줄임
overlay_indices = np.random.choice(len(histograms), n_overlay, replace=False)
for idx in overlay_indices:
    ax2.plot(bin_centers, histograms[idx], alpha=0.3, linewidth=0.8)
ax2.plot(bin_centers, mean_histogram, color='red', linewidth=2.5, label='평균 분포')
ax2.fill_between(bin_centers, mean_histogram - std_histogram, mean_histogram + std_histogram, 
                  color='red', alpha=0.2, label='±1 표준편차')
ax2.set_xlabel('BAIS2 값', fontsize=10)
ax2.set_ylabel('정규화된 빈도', fontsize=10)
ax2.set_title(f'개별 히스토그램 오버레이 ({n_overlay}개 샘플)', fontsize=12, fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(alpha=0.3)

# 4-3. 통계 박스플롯 (평균)
ax3 = fig.add_subplot(gs[1, 1])
bp1 = ax3.boxplot([df_stats['mean']], positions=[1], widths=0.6, patch_artist=True,
                    boxprops=dict(facecolor='lightcoral', alpha=0.7),
                    medianprops=dict(color='darkred', linewidth=2))
ax3.set_ylabel('BAIS2 평균', fontsize=10)
ax3.set_title('영상별 평균 분포', fontsize=12, fontweight='bold')
ax3.set_xticks([1])
ax3.set_xticklabels([f'{len(df_stats)}개 영상'])
ax3.grid(axis='y', alpha=0.3)
ax3.text(1.3, df_stats['mean'].median(), f"중앙값: {df_stats['mean'].median():.3f}\nCV: {mean_cv:.3f}", 
         fontsize=9, verticalalignment='center')

# 4-4. JS Divergence 분포
ax4 = fig.add_subplot(gs[1, 2])
ax4.hist(js_divergences, bins=20, color='orange', alpha=0.7, edgecolor='black')
ax4.set_xlabel('Jensen-Shannon Divergence', fontsize=10)
ax4.set_ylabel('빈도', fontsize=10)
ax4.set_title(f'평균 분포와의 JS Divergence ({len(js_divergences)}개 샘플)', fontsize=12, fontweight='bold')
ax4.axvline(np.mean(js_divergences), color='red', linestyle='--', linewidth=2, 
            label=f'평균: {np.mean(js_divergences):.4f}')
ax4.legend(fontsize=9)
ax4.grid(axis='y', alpha=0.3)

plt.suptitle('BAIS2 히스토그램 분포 종합 분석', fontsize=16, fontweight='bold', y=0.995)

# 저장
fig_path = os.path.join(analysis_output_dir, "histogram_analysis.png")
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
print(f"      ✅ 시각화 저장: {fig_path}")
plt.show()

# ============================================================
# 5단계: 결과 저장
# ============================================================
print("\n[5/5] 분석 결과 저장 중...")

# 5-1. 통계 요약 CSV
stats_csv = os.path.join(analysis_output_dir, "statistics_summary.csv")
df_stats.to_csv(stats_csv, index=False, encoding='utf-8-sig')
print(f"      ✅ {stats_csv}")

# 5-2. 히스토그램 데이터 CSV
df_histogram = pd.DataFrame(
    histogram_counts,
    index=[os.path.basename(f) for f in tif_files],
    columns=[f'bin_{i}' for i in range(n_bins)]
)
histogram_csv = os.path.join(analysis_output_dir, "histogram_data.csv")
df_histogram.to_csv(histogram_csv, encoding='utf-8-sig')
print(f"      ✅ {histogram_csv}")

# 5-3. 분포 거리 CSV
df_distances = pd.DataFrame({
    'sample_index': sample_indices[:len(js_divergences)],
    'filename': [os.path.basename(tif_files[i]) for i in sample_indices[:len(js_divergences)]],
    'js_divergence': js_divergences
})
distances_csv = os.path.join(analysis_output_dir, "distribution_distances.csv")
df_distances.to_csv(distances_csv, index=False, encoding='utf-8-sig')
print(f"      ✅ {distances_csv}")

# 5-4. 종합 분석 리포트 TXT
report_path = os.path.join(analysis_output_dir, "analysis_report.txt")
with open(report_path, 'w', encoding='utf-8') as f:
    f.write("="*70 + "\n")
    f.write("BAIS2 히스토그램 분포 분석 리포트\n")
    f.write("="*70 + "\n\n")
    f.write(f"분석 일시: {pd.Timestamp.now()}\n")
    f.write(f"입력 경로: {input_dir}\n")
    f.write(f"총 영상 수: {len(df_stats)}개\n")
    f.write(f"총 유효 픽셀: {total_valid_pixels:,}개\n\n")
    
    f.write("[1] BAIS2 값 범위\n")
    f.write(f"  - 최소값: {df_stats['min'].min():.6f}\n")
    f.write(f"  - 최대값: {df_stats['max'].max():.6f}\n")
    f.write(f"  - 범위: {df_stats['max'].max() - df_stats['min'].min():.6f}\n\n")
    
    f.write("[2] 영상별 평균 통계\n")
    f.write(f"  - 평균의 평균: {df_stats['mean'].mean():.6f}\n")
    f.write(f"  - 평균의 표준편차: {df_stats['mean'].std():.6f}\n")
    f.write(f"  - 평균의 변동계수(CV): {mean_cv:.6f}\n")
    f.write(f"  - 평균의 범위: [{df_stats['mean'].min():.6f}, {df_stats['mean'].max():.6f}]\n\n")
    
    f.write("[3] 영상별 표준편차 통계\n")
    f.write(f"  - 표준편차의 평균: {df_stats['std'].mean():.6f}\n")
    f.write(f"  - 표준편차의 표준편차: {df_stats['std'].std():.6f}\n")
    f.write(f"  - 표준편차의 변동계수(CV): {std_cv:.6f}\n\n")
    
    f.write("[4] 분포 차이 분석\n")
    f.write(f"  - 평균 JS Divergence: {np.mean(js_divergences):.6f} ± {np.std(js_divergences):.6f}\n\n")
    
    f.write("[5] 해석\n")
    if mean_cv < 0.1:
        interpretation = "매우 균일한 분포 - 영상 간 BAIS2 분포가 매우 유사함"
    elif mean_cv < 0.3:
        interpretation = "보통 변동성 - 영상 간 BAIS2 분포에 적절한 차이가 존재함"
    else:
        interpretation = "높은 변동성 - 영상 간 BAIS2 분포가 크게 상이함. 정규화 고려 필요"
    f.write(f"  {interpretation}\n\n")
    
    f.write("[6] 권장사항\n")
    if mean_cv >= 0.3:
        f.write("  - Z-score 정규화 또는 Min-Max 정규화 적용 검토\n")
        f.write("  - 이상치(outlier) 영상 제거 검토\n")
    else:
        f.write("  - 현재 분포는 분석에 적합한 수준\n")
    f.write("  - 시계열 분석 시 계절별/연도별 분포 차이 추가 분석 권장\n")

print(f"      ✅ {report_path}")

print("\n" + "="*70)
print("=== 분석 완료 ===")
print("="*70)
print(f"\n 결과 저장 위치: {analysis_output_dir}")
print(f"\n 핵심 결과:")
print(f"   • 평균의 변동계수(CV): {mean_cv:.4f}")
if mean_cv < 0.1:
    print(f"   → 매우 균일한 분포 (정규화 불필요)")
elif mean_cv < 0.3:
    print(f"   → 보통 수준의 변동성 (정규화 선택적)")
else:
    print(f"   → 높은 변동성 (정규화 권장)")

print(f"\n   • 평균 JS Divergence: {np.mean(js_divergences):.4f}")
print(f"\n 분석 리포트를 확인하세요: {report_path}\n")


=== BAIS2 히스토그램 분포 분석 시작 ===
입력 경로: D:\Landslide\data\sentinel-2\outputs\pre\BAIS2
분석 결과 저장: D:\Landslide\data\sentinel-2\outputs\pre\BAIS2_analysis

[1/5] 데이터 수집 및 기본 통계 계산 중...
      발견된 BAIS2 파일: 128개

      메모리 효율을 위해 각 영상당 2,000개 픽셀만 샘플링합니다.



      통계 수집:   3%|▎         | 4/128 [00:09<05:04,  2.45s/it]


KeyboardInterrupt: 

In [ ]:
# ============================================================
# BAIS2 래스터 산지지역 마스킹
# ============================================================
import rasterio
import rasterio.mask
import geopandas as gpd
import numpy as np
import glob
import os
from tqdm import tqdm

# ============================================================
# 1. 기본 설정 및 경로 정의
# ============================================================
# 원본 BAIS2 래스터 파일 경로
input_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\pre\BAIS2"

# 마스킹된 결과를 저장할 경로
output_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\pre\BAIS2_masked"

# 산지지역 마스크 파일 경로
mask_path = r"D:\Landslide\data\processed\gyeongnam\terrain_information\gyeongnam_forest_mask.gpkg"

# 출력 디렉토리 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("=" * 70)
print("=== BAIS2 산지지역 마스킹 시작 ===")
print("=" * 70)
print(f"입력 경로: {input_dir}")
print(f"출력 경로: {output_dir}")
print(f"마스크 파일: {mask_path}\n")

# ============================================================
# 2. 산지지역 마스크 로드
# ============================================================
print("[1/3] 산지지역 마스크 로드 중...")

if not os.path.exists(mask_path):
    raise FileNotFoundError(f"마스크 파일을 찾을 수 없습니다: {mask_path}")

# GeoPackage 읽기
gdf_mask = gpd.read_file(mask_path)

print(f"   ✅ 마스크 로드 완료")
print(f"      피처 수: {len(gdf_mask)}개")
print(f"      CRS: {gdf_mask.crs}")
print(f"      총 면적: {gdf_mask.geometry.area.sum() / 1e6:.2f} km²\n")

# ============================================================
# 3. 모든 BAIS2 래스터 파일 마스킹
# ============================================================
print("[2/3] BAIS2 래스터 파일 마스킹 중...")

# 모든 TIF 파일 목록 가져오기
all_tif_files = sorted(glob.glob(os.path.join(input_dir, "*.tif")))
print(f"   발견된 TIF 파일: {len(all_tif_files)}개\n")

# 마스킹 결과 저장
masking_results = []
success_count = 0
failed_count = 0
failed_files = []

for idx, tif_path in enumerate(tqdm(all_tif_files, desc="   처리 중"), 1):
    filename = os.path.basename(tif_path)
    
    try:
        # 래스터 파일 열기
        with rasterio.open(tif_path) as src:
            src_crs = src.crs
            
            # 마스크를 래스터와 동일한 CRS로 변환 (필요시)
            if gdf_mask.crs != src_crs:
                gdf_mask_reprojected = gdf_mask.to_crs(src_crs)
            else:
                gdf_mask_reprojected = gdf_mask
            
            # 마스크 적용하여 래스터 자르기
            # crop=True: 마스크 영역에 맞게 래스터 크기 조정
            # all_touched=False: 폴리곤 내부에 중심이 있는 픽셀만 포함 (더 보수적)
            out_image, out_transform = rasterio.mask.mask(
                src, 
                gdf_mask_reprojected.geometry, 
                crop=True,
                all_touched=False,
                nodata=np.nan
            )
            
            # 메타데이터 업데이트
            out_meta = src.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform,
                "nodata": np.nan,
                "compress": "lzw"
            })
        
        # 마스킹 통계 계산
        total_pixels = out_image.size
        valid_pixels = np.sum(~np.isnan(out_image))
        masked_pixels = total_pixels - valid_pixels
        valid_ratio = (valid_pixels / total_pixels) * 100
        
        # 결과 저장
        output_filename = f"masked_{filename}"
        output_path = os.path.join(output_dir, output_filename)
        
        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)
            dest.set_band_description(1, 'BAIS2_Forest_Masked')
            dest.update_tags(1,
                           MASKING='Forest Area Mask Applied',
                           MASK_FILE=os.path.basename(mask_path),
                           VALID_PIXELS=str(valid_pixels),
                           MASKED_PIXELS=str(masked_pixels),
                           VALID_RATIO=f'{valid_ratio:.2f}%')
        
        # 결과 기록
        masking_results.append({
            'filename': filename,
            'total_pixels': total_pixels,
            'valid_pixels': valid_pixels,
            'masked_pixels': masked_pixels,
            'valid_ratio': valid_ratio,
            'output_file': output_filename
        })
        
        success_count += 1
        
    except Exception as e:
        error_msg = str(e)[:100]
        print(f"\n   ✗ [{idx}] {filename}: 오류 - {error_msg}")
        failed_count += 1
        failed_files.append((filename, error_msg))

print(f"\n   ✅ 마스킹 완료: {success_count}개 성공, {failed_count}개 실패\n")

# ============================================================
# 4. 결과 요약 및 저장
# ============================================================
print("[3/3] 결과 요약 및 저장 중...")

# 마스킹 결과 DataFrame
df_masking = pd.DataFrame(masking_results)

if len(df_masking) > 0:
    # 통계 요약
    print(f"\n   [마스킹 통계]")
    print(f"   - 평균 유효 픽셀 비율: {df_masking['valid_ratio'].mean():.2f}%")
    print(f"   - 총 유효 픽셀: {df_masking['valid_pixels'].sum():,}개")
    print(f"   - 총 마스킹된 픽셀: {df_masking['masked_pixels'].sum():,}개")
    
    # CSV 저장
    masking_csv = os.path.join(output_dir, "masking_results.csv")
    df_masking.to_csv(masking_csv, index=False, encoding='utf-8-sig')
    print(f"\n   ✅ 마스킹 결과 저장: {masking_csv}")

# 실패 목록 저장
if failed_files:
    df_failed = pd.DataFrame(failed_files, columns=['filename', 'error'])
    failed_csv = os.path.join(output_dir, "failed_files.csv")
    df_failed.to_csv(failed_csv, index=False, encoding='utf-8-sig')
    print(f"   ✅ 실패 목록 저장: {failed_csv}")

# 최종 요약 리포트
report_path = os.path.join(output_dir, "masking_report.txt")
with open(report_path, 'w', encoding='utf-8') as f:
    f.write("="*70 + "\n")
    f.write("BAIS2 산지지역 마스킹 리포트\n")
    f.write("="*70 + "\n\n")
    f.write(f"작업 일시: {pd.Timestamp.now()}\n")
    f.write(f"입력 경로: {input_dir}\n")
    f.write(f"출력 경로: {output_dir}\n")
    f.write(f"마스크 파일: {mask_path}\n\n")
    
    f.write(f"[마스크 정보]\n")
    f.write(f"  피처 수: {len(gdf_mask)}개\n")
    f.write(f"  CRS: {gdf_mask.crs}\n")
    f.write(f"  총 면적: {gdf_mask.geometry.area.sum() / 1e6:.2f} km²\n\n")
    
    f.write(f"[처리 결과]\n")
    f.write(f"  총 파일 수: {len(all_tif_files)}개\n")
    f.write(f"  마스킹 완료: {success_count}개\n")
    f.write(f"  실패: {failed_count}개\n\n")
    
    if len(df_masking) > 0:
        f.write(f"[마스킹 통계]\n")
        f.write(f"  평균 유효 픽셀 비율: {df_masking['valid_ratio'].mean():.2f}%\n")
        f.write(f"  총 유효 픽셀: {df_masking['valid_pixels'].sum():,}개\n")
        f.write(f"  총 마스킹된 픽셀: {df_masking['masked_pixels'].sum():,}개\n\n")
    
    if failed_files:
        f.write(f"[실패한 파일]\n")
        for filename, error in failed_files[:20]:
            f.write(f"  - {filename}: {error}\n")
        if len(failed_files) > 20:
            f.write(f"  ... 외 {len(failed_files)-20}개\n")

print(f"   ✅ 리포트 저장: {report_path}")

print("\n" + "="*70)
print("=== 마스킹 완료 ===")
print("="*70)
print(f"\n 총 {success_count}개 파일이 마스킹되어 저장되었습니다.")
print(f" 출력 경로: {output_dir}")

=== BAIS2 산지지역 마스킹 시작 ===
입력 경로: D:\Landslide\data\sentinel-2\outputs\pre\BAIS2
출력 경로: D:\Landslide\data\sentinel-2\outputs\pre\BAIS2_masked
마스크 파일: D:\Landslide\data\gyeongnam\terrain_information\gyeongnam_forest_mask.gpkg

[1/3] 산지지역 마스크 로드 중...
   ✅ 마스크 로드 완료
      피처 수: 1개
      CRS: EPSG:5179
      총 면적: 6897.18 km²

[2/3] BAIS2 래스터 파일 마스킹 중...
   발견된 TIF 파일: 128개



   처리 중: 100%|██████████| 128/128 [1:10:47<00:00, 33.18s/it]


   ✅ 마스킹 완료: 128개 성공, 0개 실패

[3/3] 결과 요약 및 저장 중...

   [마스킹 통계]
   - 평균 유효 픽셀 비율: 32.54%
   - 총 유효 픽셀: 676,996,838개
   - 총 마스킹된 픽셀: 1,511,470,267개

   ✅ 마스킹 결과 저장: D:\Landslide\data\sentinel-2\outputs\pre\BAIS2_masked\masking_results.csv
   ✅ 리포트 저장: D:\Landslide\data\sentinel-2\outputs\pre\BAIS2_masked\masking_report.txt

=== 마스킹 완료 ===

 총 128개 파일이 마스킹되어 저장되었습니다.
 출력 경로: D:\Landslide\data\sentinel-2\outputs\pre\BAIS2_masked


In [ ]:
# dBAIS2 계산 셀
import rasterio
import numpy as np
import pandas as pd
import glob
import os
import re
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.merge import merge
from tqdm import tqdm

reference_csv = r"D:\Landslide\data\raw\산불발생이력\s2_matches-15.csv"
pre_id_column = 'pre_product_id'
post_id_column = 'post_product_id'

pre_masked_BAIS2_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\pre\BAIS2_masked"
post_masked_BAIS2_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\post\BAIS2_masked"

output_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\dBAIS2"
unpaired_log = os.path.join(output_dir, "unpaired.log")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("="*70)
print("=== dBAIS2 계산 시작 ===")
print("="*70)
print(f"CSV 파일: {reference_csv}")
print(f"Pre-BAIS2 경로: {pre_masked_BAIS2_dir}")
print(f"Post-BAIS2 경로: {post_masked_BAIS2_dir}")
print(f"출력 경로: {output_dir}\n")

# ============================================================
# 1단계: CSV 파일 로드 및 Pair 리스트 생성
# ============================================================
print("[1/4] CSV 파일 로드 및 Pair 리스트 생성 중...")

df_ref = pd.read_csv(reference_csv, encoding='utf-8-sig')
print(f"   총 레코드 수: {len(df_ref)}개")

# 짧은 파일명 추출 함수 (20151028_R103_T52SCD 형식으로 변환)
def extract_short_name(product_id):
    """
    S2A_MSIL2A_20151028T020802_N0500_R103_T52SCD_20231009T132010
    -> 20151028_R103_T52SCD
    """
    if pd.isna(product_id) or product_id == '':
        return None

    match = re.search(r'(\d{8})T\d+_N\d+_(R\d+)_(T\w+)_', str(product_id))
    if match:
        date = match.group(1)
        orbit = match.group(2)
        tile = match.group(3)
        return f"{date}_{orbit}_{tile}"
    return None

# 파일명 디렉토리 매핑 생성
def build_file_mapping(directory):
    """디렉토리 내 파일명 → 전체 경로 매핑
    
    다음 패턴들을 모두 지원:
    1. masked_20151028_R103_T52SCD_BAIS2.tif (masked 버전)
    2. S2A_MSIL2A_20151028T020802_N0500_R103_T52SCD_20231009T132010_BAIS2.tif (원본 버전)
    3. 20151028_R103_T52SCD_BAIS2.tif (prefix 없는 버전)
    """
    file_map = {}
    if not os.path.exists(directory):
        print(f"   경고: 경로가 존재하지 않습니다: {directory}")
        return file_map

    for tif_file in glob.glob(os.path.join(directory, "*_BAIS2.tif")):
        basename = os.path.basename(tif_file)
        
        # 패턴 1: masked_YYYYMMDD_RXXX_TXXXXX_BAIS2.tif
        if basename.startswith("masked_"):
            match = re.search(r'masked_(\d{8}_R\d+_T\w+)_BAIS2\.tif', basename)
            if match:
                short_name = match.group(1)
                file_map[short_name] = tif_file
                continue
        
        # 패턴 2: S2X_MSIL2A_YYYYMMDDTHHMMSS_NXXXX_RXXX_TXXXXX_YYYYMMDDTHHMMSS_BAIS2.tif
        match = re.search(r'S2[AB]_MSIL2A_(\d{8})T\d+_N\d+_(R\d+)_(T\w+)_\d+T\d+_BAIS2\.tif', basename)
        if match:
            date = match.group(1)
            orbit = match.group(2)
            tile = match.group(3)
            short_name = f"{date}_{orbit}_{tile}"
            file_map[short_name] = tif_file
            continue
        
        # 패턴 3: YYYYMMDD_RXXX_TXXXXX_BAIS2.tif (prefix 없음)
        match = re.search(r'^(\d{8}_R\d+_T\w+)_BAIS2\.tif$', basename)
        if match:
            short_name = match.group(1)
            file_map[short_name] = tif_file

    return file_map

pre_file_map = build_file_mapping(pre_masked_BAIS2_dir)
post_file_map = build_file_mapping(post_masked_BAIS2_dir)

print(f"   Pre-BAIS2 파일: {len(pre_file_map)}개")
print(f"   Post-BAIS2 파일: {len(post_file_map)}개")

# Pair 리스트 생성
pair_list = []
unpaired_cases = []

for idx, row in df_ref.iterrows():
    fire_id = row['fire_id']
    fire_date = row['fire_end_date']
    pre_product = row.get(pre_id_column)
    post_product = row.get(post_id_column)

    pre_short = extract_short_name(pre_product)
    post_short = extract_short_name(post_product)

    # Pair 유효성 체크
    has_pre = pre_short is not None and pre_short in pre_file_map
    has_post = post_short is not None and post_short in post_file_map

    if has_pre and has_post:
        # 유효한 Pair
        pair_list.append({
            'fire_id': fire_id,
            'fire_date': fire_date,
            'pre_short_name': pre_short,
            'post_short_name': post_short,
            'pre_file': pre_file_map[pre_short],
            'post_file': post_file_map[post_short]
        })
    else:
        # Unpaired 케이스
        reason = []
        if pre_short is None:
            reason.append("pre_product_id 없음")
        elif pre_short not in pre_file_map:
            reason.append(f"pre 파일 없음 ({pre_short})")

        if post_short is None:
            reason.append("post_product_id 없음")
        elif post_short not in post_file_map:
            reason.append(f"post 파일 없음 ({post_short})")

        unpaired_cases.append({
            'fire_id': fire_id,
            'fire_date': fire_date,
            'pre_product_id': pre_product,
            'post_product_id': post_product,
            'reason': '; '.join(reason)
        })

print(f"\n   완료: Pair 리스트 생성")
print(f"      유효한 Pair: {len(pair_list)}개")
print(f"      Unpaired: {len(unpaired_cases)}개\n")

# Unpaired 케이스 로그 저장
if unpaired_cases:
    with open(unpaired_log, 'w', encoding='utf-8') as f:
        f.write("="*70 + "\n")
        f.write("Unpaired 산불-영상 케이스 로그\n")
        f.write("="*70 + "\n\n")
        f.write(f"작성 일시: {pd.Timestamp.now()}\n")
        f.write(f"총 Unpaired 케이스: {len(unpaired_cases)}개\n\n")

        for case in unpaired_cases:
            f.write(f"[Fire ID: {case['fire_id']}]\n")
            f.write(f"  산불 진화일: {case['fire_date']}\n")
            f.write(f"  Pre-Product ID: {case['pre_product_id']}\n")
            f.write(f"  Post-Product ID: {case['post_product_id']}\n")
            f.write(f"  사유: {case['reason']}\n\n")

    print(f"   Unpaired 로그 저장: {unpaired_log}\n")

if len(pair_list) == 0:
    print("오류: 처리할 유효한 Pair가 없습니다.")
    raise SystemExit("No valid pairs to process")

# ============================================================
# 2단계: dBAIS2 계산 함수 정의
# ============================================================
print("[2/4] dBAIS2 계산 함수 정의...")

def calculate_dbais2_intersection(pre_file, post_file, fire_id, pre_name, post_name, output_dir):
    """
    두 BAIS2 영상의 겹치는 영역(intersection)에 대해 dBAIS2 계산

    Parameters:
        pre_file: Pre-fire BAIS2 파일 경로
        post_file: Post-fire BAIS2 파일 경로
        fire_id: 산불 ID
        pre_name: Pre 짧은 이름
        post_name: Post 짧은 이름
        output_dir: 출력 디렉토리

    Returns:
        success: 성공 여부 (bool)
        message: 결과 메시지
    """
    try:
        # Pre/Post 파일 열기
        with rasterio.open(pre_file) as pre_src, rasterio.open(post_file) as post_src:

            # 두 영상의 경계(bounds) 가져오기
            pre_bounds = pre_src.bounds
            post_bounds = post_src.bounds

            # Intersection (겹치는 영역) 계산
            inter_left = max(pre_bounds.left, post_bounds.left)
            inter_bottom = max(pre_bounds.bottom, post_bounds.bottom)
            inter_right = min(pre_bounds.right, post_bounds.right)
            inter_top = min(pre_bounds.top, post_bounds.top)

            # 겹치는 영역이 있는지 확인
            if inter_left >= inter_right or inter_bottom >= inter_top:
                return False, "No intersection between pre and post images"

            inter_bounds = (inter_left, inter_bottom, inter_right, inter_top)

            # CRS가 다른 경우 처리
            if pre_src.crs != post_src.crs:
                # Post를 Pre의 CRS로 변환
                post_crs = pre_src.crs
            else:
                post_crs = pre_src.crs

            # Pre 영상: intersection 영역만 읽기 (window 사용)
            pre_window = pre_src.window(*inter_bounds)
            pre_data = pre_src.read(1, window=pre_window).astype(np.float32)
            pre_transform = pre_src.window_transform(pre_window)

            # Post 영상: intersection 영역만 읽기 및 리프로젝션
            post_window = post_src.window(*inter_bounds)
            post_data_raw = post_src.read(1, window=post_window).astype(np.float32)
            post_transform_raw = post_src.window_transform(post_window)

            # Post를 Pre 해상도/그리드로 맞추기 (reproject)
            post_data = np.empty_like(pre_data)
            reproject(
                source=post_data_raw,
                destination=post_data,
                src_transform=post_transform_raw,
                src_crs=post_src.crs,
                dst_transform=pre_transform,
                dst_crs=post_crs,
                resampling=Resampling.bilinear,
                src_nodata=np.nan,
                dst_nodata=np.nan
            )

            # dBAIS2 계산: POST - PRE
            dbais2 = post_data - pre_data

            # 유효 픽셀 마스크 (Pre와 Post 모두 유효한 픽셀만)
            valid_mask = ~np.isnan(pre_data) & ~np.isnan(post_data)
            dbais2[~valid_mask] = np.nan

            # 통계 계산
            valid_pixels = np.sum(valid_mask)
            if valid_pixels == 0:
                return False, "No valid pixels in intersection"

            dbais2_min = np.nanmin(dbais2)
            dbais2_max = np.nanmax(dbais2)
            dbais2_mean = np.nanmean(dbais2)
            dbais2_std = np.nanstd(dbais2)

            # 출력 파일명 생성
            output_filename = f"dBAIS2_fire{fire_id:04d}_{pre_name}_to_{post_name}.tif"
            output_path = os.path.join(output_dir, output_filename)

            # 메타데이터 설정
            profile = pre_src.profile.copy()
            profile.update({
                'driver': 'GTiff',
                'dtype': rasterio.float32,
                'count': 1,
                'width': pre_data.shape[1],
                'height': pre_data.shape[0],
                'crs': post_crs,
                'transform': pre_transform,
                'compress': 'lzw',
                'nodata': np.nan
            })

            # 저장
            with rasterio.open(output_path, 'w', **profile) as dst:
                dst.write(dbais2, 1)
                dst.set_band_description(1, 'dBAIS2 (Post - Pre)')
                dst.update_tags(1,
                               FIRE_ID=str(fire_id),
                               PRE_IMAGE=pre_name,
                               POST_IMAGE=post_name,
                               FORMULA='POST_BAIS2 - PRE_BAIS2',
                               VALID_PIXELS=str(valid_pixels),
                               DBAIS2_MIN=f'{dbais2_min:.6f}',
                               DBAIS2_MAX=f'{dbais2_max:.6f}',
                               DBAIS2_MEAN=f'{dbais2_mean:.6f}',
                               DBAIS2_STD=f'{dbais2_std:.6f}')

            result_msg = (f"OK: dBAIS2: [{dbais2_min:.3f}, {dbais2_max:.3f}], "
                         f"Mean: {dbais2_mean:.3f}, Valid: {valid_pixels:,}px")

            return True, result_msg

    except Exception as e:
        error_msg = f"Error: {str(e)[:100]}"
        return False, error_msg

print("   완료: 함수 정의\n")

# ============================================================
# 3단계: dBAIS2 계산 수행
# ============================================================
print(f"[3/4] dBAIS2 계산 수행 중 ({len(pair_list)}개 Pair)...\n")

results = []
success_count = 0
fail_count = 0

for pair in tqdm(pair_list, desc="   처리 중"):
    fire_id = pair['fire_id']
    pre_file = pair['pre_file']
    post_file = pair['post_file']
    pre_name = pair['pre_short_name']
    post_name = pair['post_short_name']

    success, message = calculate_dbais2_intersection(
        pre_file, post_file, fire_id, pre_name, post_name, output_dir
    )

    if success:
        success_count += 1
    else:
        fail_count += 1

    results.append({
        'fire_id': fire_id,
        'fire_date': pair['fire_date'],
        'pre_name': pre_name,
        'post_name': post_name,
        'success': success,
        'message': message
    })

print(f"\n   완료: 계산 완료 - {success_count}개 성공, {fail_count}개 실패\n")

# ============================================================
# 4단계: 결과 저장
# ============================================================
print("[4/4] 결과 저장 중...")

df_results = pd.DataFrame(results)

# 결과 CSV 저장
results_csv = os.path.join(output_dir, "dbais2_results.csv")
df_results.to_csv(results_csv, index=False, encoding='utf-8-sig')
print(f"   완료: 결과 CSV 저장 - {results_csv}")

# 최종 리포트 저장
report_path = os.path.join(output_dir, "dbais2_report.txt")
with open(report_path, 'w', encoding='utf-8') as f:
    f.write("="*70 + "\n")
    f.write("dBAIS2 계산 리포트\n")
    f.write("="*70 + "\n\n")
    f.write(f"작업 일시: {pd.Timestamp.now()}\n")
    f.write(f"CSV 파일: {reference_csv}\n")
    f.write(f"출력 경로: {output_dir}\n\n")

    f.write(f"[처리 결과]\n")
    f.write(f"  총 레코드 수: {len(df_ref)}개\n")
    f.write(f"  유효한 Pair: {len(pair_list)}개\n")
    f.write(f"  Unpaired: {len(unpaired_cases)}개\n")
    f.write(f"  dBAIS2 계산 성공: {success_count}개\n")
    f.write(f"  dBAIS2 계산 실패: {fail_count}개\n\n")

    if fail_count > 0:
        f.write(f"[실패한 케이스]\n")
        failed = df_results[df_results['success'] == False]
        for _, row in failed.iterrows():
            f.write(f"  Fire ID {row['fire_id']}: {row['message']}\n")

print(f"   완료: 리포트 저장 - {report_path}")

print("\n" + "="*70)
print("=== dBAIS2 계산 완료 ===")
print("="*70)
print(f"\n 총 {success_count}개 dBAIS2 파일이 생성되었습니다.")
print(f" 출력 경로: {output_dir}")
print(f"\n 주요 파일:")
print(f"   - dBAIS2 결과: {results_csv}")
print(f"   - 처리 리포트: {report_path}")
if unpaired_cases:
    print(f"   - Unpaired 로그: {unpaired_log}")
print()


In [ ]:
# RdBAIS2 계산 셀
import rasterio
import numpy as np
import pandas as pd
import glob
import os
import re
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.merge import merge
from tqdm import tqdm

reference_csv = r"D:\Landslide\data\raw\산불발생이력\s2_matches-15.csv"
pre_id_column = 'pre_product_id'
post_id_column = 'post_product_id'

pre_masked_BAIS2_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\pre\BAIS2_masked"
post_masked_BAIS2_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\post\BAIS2_masked"

output_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\RdBAIS2"
unpaired_log = os.path.join(output_dir, "unpaired.log")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("="*70)
print("=== RdBAIS2 계산 시작 ===")
print("="*70)
print(f"CSV 파일: {reference_csv}")
print(f"Pre-BAIS2 경로: {pre_masked_BAIS2_dir}")
print(f"Post-BAIS2 경로: {post_masked_BAIS2_dir}")
print(f"출력 경로: {output_dir}\n")

# ============================================================
# 1단계: CSV 파일 로드 및 Pair 리스트 생성
# ============================================================
print("[1/4] CSV 파일 로드 및 Pair 리스트 생성 중...")

df_ref = pd.read_csv(reference_csv, encoding='utf-8-sig')
print(f"   총 레코드 수: {len(df_ref)}개")

# 짧은 파일명 추출 함수 (20151028_R103_T52SCD 형식으로 변환)
def extract_short_name(product_id):
    """
    S2A_MSIL2A_20151028T020802_N0500_R103_T52SCD_20231009T132010
    -> 20151028_R103_T52SCD
    """
    if pd.isna(product_id) or product_id == '':
        return None

    match = re.search(r'(\d{8})T\d+_N\d+_(R\d+)_(T\w+)_', str(product_id))
    if match:
        date = match.group(1)
        orbit = match.group(2)
        tile = match.group(3)
        return f"{date}_{orbit}_{tile}"
    return None

# 파일명 디렉토리 매핑 생성
def build_file_mapping(directory):
    """디렉토리 내 파일명 → 전체 경로 매핑
    
    다음 패턴들을 모두 지원:
    1. masked_20151028_R103_T52SCD_BAIS2.tif (masked 버전)
    2. S2A_MSIL2A_20151028T020802_N0500_R103_T52SCD_20231009T132010_BAIS2.tif (원본 버전)
    3. 20151028_R103_T52SCD_BAIS2.tif (prefix 없는 버전)
    """
    file_map = {}
    if not os.path.exists(directory):
        print(f"   경고: 경로가 존재하지 않습니다: {directory}")
        return file_map

    for tif_file in glob.glob(os.path.join(directory, "*_BAIS2.tif")):
        basename = os.path.basename(tif_file)
        
        # 패턴 1: masked_YYYYMMDD_RXXX_TXXXXX_BAIS2.tif
        if basename.startswith("masked_"):
            match = re.search(r'masked_(\d{8}_R\d+_T\w+)_BAIS2\.tif', basename)
            if match:
                short_name = match.group(1)
                file_map[short_name] = tif_file
                continue
        
        # 패턴 2: S2X_MSIL2A_YYYYMMDDTHHMMSS_NXXXX_RXXX_TXXXXX_YYYYMMDDTHHMMSS_BAIS2.tif
        match = re.search(r'S2[AB]_MSIL2A_(\d{8})T\d+_N\d+_(R\d+)_(T\w+)_\d+T\d+_BAIS2\.tif', basename)
        if match:
            date = match.group(1)
            orbit = match.group(2)
            tile = match.group(3)
            short_name = f"{date}_{orbit}_{tile}"
            file_map[short_name] = tif_file
            continue
        
        # 패턴 3: YYYYMMDD_RXXX_TXXXXX_BAIS2.tif (prefix 없음)
        match = re.search(r'^(\d{8}_R\d+_T\w+)_BAIS2\.tif$', basename)
        if match:
            short_name = match.group(1)
            file_map[short_name] = tif_file

    return file_map

pre_file_map = build_file_mapping(pre_masked_BAIS2_dir)
post_file_map = build_file_mapping(post_masked_BAIS2_dir)

print(f"   Pre-BAIS2 파일: {len(pre_file_map)}개")
print(f"   Post-BAIS2 파일: {len(post_file_map)}개")

# Pair 리스트 생성
pair_list = []
unpaired_cases = []

for idx, row in df_ref.iterrows():
    fire_id = row['fire_id']
    fire_date = row['fire_end_date']
    pre_product = row.get(pre_id_column)
    post_product = row.get(post_id_column)

    pre_short = extract_short_name(pre_product)
    post_short = extract_short_name(post_product)

    # Pair 유효성 체크
    has_pre = pre_short is not None and pre_short in pre_file_map
    has_post = post_short is not None and post_short in post_file_map

    if has_pre and has_post:
        # 유효한 Pair
        pair_list.append({
            'fire_id': fire_id,
            'fire_date': fire_date,
            'pre_short_name': pre_short,
            'post_short_name': post_short,
            'pre_file': pre_file_map[pre_short],
            'post_file': post_file_map[post_short]
        })
    else:
        # Unpaired 케이스
        reason = []
        if pre_short is None:
            reason.append("pre_product_id 없음")
        elif pre_short not in pre_file_map:
            reason.append(f"pre 파일 없음 ({pre_short})")

        if post_short is None:
            reason.append("post_product_id 없음")
        elif post_short not in post_file_map:
            reason.append(f"post 파일 없음 ({post_short})")

        unpaired_cases.append({
            'fire_id': fire_id,
            'fire_date': fire_date,
            'pre_product_id': pre_product,
            'post_product_id': post_product,
            'reason': '; '.join(reason)
        })

print(f"\n   완료: Pair 리스트 생성")
print(f"      유효한 Pair: {len(pair_list)}개")
print(f"      Unpaired: {len(unpaired_cases)}개\n")

# Unpaired 케이스 로그 저장
if unpaired_cases:
    with open(unpaired_log, 'w', encoding='utf-8') as f:
        f.write("="*70 + "\n")
        f.write("Unpaired 산불-영상 케이스 로그\n")
        f.write("="*70 + "\n\n")
        f.write(f"작성 일시: {pd.Timestamp.now()}\n")
        f.write(f"총 Unpaired 케이스: {len(unpaired_cases)}개\n\n")

        for case in unpaired_cases:
            f.write(f"[Fire ID: {case['fire_id']}]\n")
            f.write(f"  산불 진화일: {case['fire_date']}\n")
            f.write(f"  Pre-Product ID: {case['pre_product_id']}\n")
            f.write(f"  Post-Product ID: {case['post_product_id']}\n")
            f.write(f"  사유: {case['reason']}\n\n")

    print(f"   Unpaired 로그 저장: {unpaired_log}\n")

if len(pair_list) == 0:
    print("오류: 처리할 유효한 Pair가 없습니다.")
    raise SystemExit("No valid pairs to process")

# ============================================================
# 2단계: RdBAIS2 계산 함수 정의
# ============================================================
print("[2/4] RdBAIS2 계산 함수 정의...")

def calculate_rdbais2_intersection(pre_file, post_file, fire_id, pre_name, post_name, output_dir):
    """
    두 BAIS2 영상의 겹치는 영역(intersection)에 대해 RdBAIS2 계산
    
    RdBAIS2 (Relativized dBAIS2) = dBAIS2 / sqrt(|preBAIS2| + 1)
    - 산불 전 식생 상태를 고려한 상대적 피해도 지표
    - BAIS2는 음수 값을 가질 수 있으므로 +1을 더하여 항상 양수로 만듦
    - preBAIS2 값이 낮을수록 더 큰 가중치 적용

    Parameters:
        pre_file: Pre-fire BAIS2 파일 경로
        post_file: Post-fire BAIS2 파일 경로
        fire_id: 산불 ID
        pre_name: Pre 짧은 이름
        post_name: Post 짧은 이름
        output_dir: 출력 디렉토리

    Returns:
        success: 성공 여부 (bool)
        message: 결과 메시지
    """
    try:
        # Pre/Post 파일 열기
        with rasterio.open(pre_file) as pre_src, rasterio.open(post_file) as post_src:

            # 두 영상의 경계(bounds) 가져오기
            pre_bounds = pre_src.bounds
            post_bounds = post_src.bounds

            # Intersection (겹치는 영역) 계산
            inter_left = max(pre_bounds.left, post_bounds.left)
            inter_bottom = max(pre_bounds.bottom, post_bounds.bottom)
            inter_right = min(pre_bounds.right, post_bounds.right)
            inter_top = min(pre_bounds.top, post_bounds.top)

            # 겹치는 영역이 있는지 확인
            if inter_left >= inter_right or inter_bottom >= inter_top:
                return False, "No intersection between pre and post images"

            inter_bounds = (inter_left, inter_bottom, inter_right, inter_top)

            # CRS가 다른 경우 처리
            if pre_src.crs != post_src.crs:
                # Post를 Pre의 CRS로 변환
                post_crs = pre_src.crs
            else:
                post_crs = pre_src.crs

            # Pre 영상: intersection 영역만 읽기 (window 사용)
            pre_window = pre_src.window(*inter_bounds)
            pre_data = pre_src.read(1, window=pre_window).astype(np.float32)
            pre_transform = pre_src.window_transform(pre_window)

            # Post 영상: intersection 영역만 읽기 및 리프로젝션
            post_window = post_src.window(*inter_bounds)
            post_data_raw = post_src.read(1, window=post_window).astype(np.float32)
            post_transform_raw = post_src.window_transform(post_window)

            # Post를 Pre 해상도/그리드로 맞추기 (reproject)
            post_data = np.empty_like(pre_data)
            reproject(
                source=post_data_raw,
                destination=post_data,
                src_transform=post_transform_raw,
                src_crs=post_src.crs,
                dst_transform=pre_transform,
                dst_crs=post_crs,
                resampling=Resampling.bilinear,
                src_nodata=np.nan,
                dst_nodata=np.nan
            )

            # dBAIS2 계산: POST - PRE
            dbais2 = post_data - pre_data

            # RdBAIS2 계산: dBAIS2 / sqrt(|preBAIS2| + 1)
            # BAIS2는 음수 값을 가질 수 있으므로 절댓값에 1을 더함
            # epsilon을 사용하여 0으로 나누는 것 방지
            epsilon = 1e-6
            pre_bais2_abs_plus_one = np.abs(pre_data) + 1.0
            
            # 안전한 분모 계산
            denominator = np.sqrt(pre_bais2_abs_plus_one)
            denominator_safe = np.where(denominator < epsilon, epsilon, denominator)
            
            # RdBAIS2 계산
            rdbais2 = dbais2 / denominator_safe

            # 유효 픽셀 마스크 (Pre와 Post 모두 유효한 픽셀만)
            valid_mask = ~np.isnan(pre_data) & ~np.isnan(post_data) & np.isfinite(rdbais2)
            rdbais2[~valid_mask] = np.nan

            # 통계 계산
            valid_pixels = np.sum(valid_mask)
            if valid_pixels == 0:
                return False, "No valid pixels in intersection"

            rdbais2_min = np.nanmin(rdbais2)
            rdbais2_max = np.nanmax(rdbais2)
            rdbais2_mean = np.nanmean(rdbais2)
            rdbais2_std = np.nanstd(rdbais2)

            # 출력 파일명 생성
            output_filename = f"RdBAIS2_fire{fire_id:04d}_{pre_name}_to_{post_name}.tif"
            output_path = os.path.join(output_dir, output_filename)

            # 메타데이터 설정
            profile = pre_src.profile.copy()
            profile.update({
                'driver': 'GTiff',
                'dtype': rasterio.float32,
                'count': 1,
                'width': pre_data.shape[1],
                'height': pre_data.shape[0],
                'crs': post_crs,
                'transform': pre_transform,
                'compress': 'lzw',
                'nodata': np.nan
            })

            # 저장
            with rasterio.open(output_path, 'w', **profile) as dst:
                dst.write(rdbais2, 1)
                dst.set_band_description(1, 'RdBAIS2 (Relativized dBAIS2)')
                dst.update_tags(1,
                               FIRE_ID=str(fire_id),
                               PRE_IMAGE=pre_name,
                               POST_IMAGE=post_name,
                               FORMULA='dBAIS2 / sqrt(|preBAIS2| + 1)',
                               VALID_PIXELS=str(valid_pixels),
                               RDBAIS2_MIN=f'{rdbais2_min:.6f}',
                               RDBAIS2_MAX=f'{rdbais2_max:.6f}',
                               RDBAIS2_MEAN=f'{rdbais2_mean:.6f}',
                               RDBAIS2_STD=f'{rdbais2_std:.6f}')

            result_msg = (f"OK: RdBAIS2: [{rdbais2_min:.3f}, {rdbais2_max:.3f}], "
                         f"Mean: {rdbais2_mean:.3f}, Valid: {valid_pixels:,}px")

            return True, result_msg

    except Exception as e:
        error_msg = f"Error: {str(e)[:100]}"
        return False, error_msg

print("   완료: 함수 정의\n")

# ============================================================
# 3단계: RdBAIS2 계산 수행
# ============================================================
print(f"[3/4] RdBAIS2 계산 수행 중 ({len(pair_list)}개 Pair)...\n")

results = []
success_count = 0
fail_count = 0

for pair in tqdm(pair_list, desc="   처리 중"):
    fire_id = pair['fire_id']
    pre_file = pair['pre_file']
    post_file = pair['post_file']
    pre_name = pair['pre_short_name']
    post_name = pair['post_short_name']

    success, message = calculate_rdbais2_intersection(
        pre_file, post_file, fire_id, pre_name, post_name, output_dir
    )

    if success:
        success_count += 1
    else:
        fail_count += 1

    results.append({
        'fire_id': fire_id,
        'fire_date': pair['fire_date'],
        'pre_name': pre_name,
        'post_name': post_name,
        'success': success,
        'message': message
    })

print(f"\n   완료: 계산 완료 - {success_count}개 성공, {fail_count}개 실패\n")

# ============================================================
# 4단계: 결과 저장
# ============================================================
print("[4/4] 결과 저장 중...")

df_results = pd.DataFrame(results)

# 결과 CSV 저장
results_csv = os.path.join(output_dir, "rdbais2_results.csv")
df_results.to_csv(results_csv, index=False, encoding='utf-8-sig')
print(f"   완료: 결과 CSV 저장 - {results_csv}")

# 최종 리포트 저장
report_path = os.path.join(output_dir, "rdbais2_report.txt")
with open(report_path, 'w', encoding='utf-8') as f:
    f.write("="*70 + "\n")
    f.write("RdBAIS2 계산 리포트\n")
    f.write("="*70 + "\n\n")
    f.write(f"작업 일시: {pd.Timestamp.now()}\n")
    f.write(f"CSV 파일: {reference_csv}\n")
    f.write(f"출력 경로: {output_dir}\n\n")

    f.write(f"[처리 결과]\n")
    f.write(f"  총 레코드 수: {len(df_ref)}개\n")
    f.write(f"  유효한 Pair: {len(pair_list)}개\n")
    f.write(f"  Unpaired: {len(unpaired_cases)}개\n")
    f.write(f"  RdBAIS2 계산 성공: {success_count}개\n")
    f.write(f"  RdBAIS2 계산 실패: {fail_count}개\n\n")

    if fail_count > 0:
        f.write(f"[실패한 케이스]\n")
        failed = df_results[df_results['success'] == False]
        for _, row in failed.iterrows():
            f.write(f"  Fire ID {row['fire_id']}: {row['message']}\n")

print(f"   완료: 리포트 저장 - {report_path}")

print("\n" + "="*70)
print("=== RdBAIS2 계산 완료 ===")
print("="*70)
print(f"\n 총 {success_count}개 RdBAIS2 파일이 생성되었습니다.")
print(f" 출력 경로: {output_dir}")
print(f"\n 주요 파일:")
print(f"   - RdBAIS2 결과: {results_csv}")
print(f"   - 처리 리포트: {report_path}")
if unpaired_cases:
    print(f"   - Unpaired 로그: {unpaired_log}")
print()
